In [1]:
# Import dependencies
import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# from prophet import Prophet

In [3]:
# Load the data from Resources folder
df = pd.read_csv('Resources/final_df.csv')
df.head()


,cmplnt_fr_dt,cmplnt_fr_tm,ofns_desc
0,2010-10-27T00:00:00.000,19:30:00,PETIT LARCENY
1,2021-05-24T00:00:00.000,03:00:00,PETIT LARCENY
2,2010-02-10T00:00:00.000,12:00:00,PETIT LARCENY
3,2010-02-09T00:00:00.000,15:00:00,PETIT LARCENY
4,2010-07-29T00:00:00.000,08:00:00,PETIT LARCENY


In [7]:
crime_df = df
crime_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6230872 entries, 0 to 6230871
Data columns (total 3 columns):
 #   Column        Dtype 
---  ------        ----- 
 0   cmplnt_fr_dt  object
 1   cmplnt_fr_tm  object
 2   ofns_desc     object
dtypes: object(3)
memory usage: 142.6+ MB


In [15]:
# Ažuriranje koda za pravilno parsiranje datuma i vremena sa milisekundama
crime_df['datetime'] = pd.to_datetime(crime_df['cmplnt_fr_dt'].str.replace('T', ' ') + ' ' + crime_df['cmplnt_fr_tm'], errors='coerce', format='%Y-%m-%d %H:%M:%S.%f')

# Provera da li je konverzija uspešna
print(crime_df['datetime'].head())


0   NaT
1   NaT
2   NaT
3   NaT
4   NaT
Name: datetime, dtype: datetime64[ns]


In [12]:
crime_df.head()

,cmplnt_fr_dt,cmplnt_fr_tm,ofns_desc,datetime
0,2010-10-27T00:00:00.000,19:30:00,PETIT LARCENY,NaT
1,2021-05-24T00:00:00.000,03:00:00,PETIT LARCENY,NaT
2,2010-02-10T00:00:00.000,12:00:00,PETIT LARCENY,NaT
3,2010-02-09T00:00:00.000,15:00:00,PETIT LARCENY,NaT
4,2010-07-29T00:00:00.000,08:00:00,PETIT LARCENY,NaT


In [13]:
# How many Nat values in datetime column
crime_df['datetime'].isna().sum()

6230872

In [18]:
# Filtriranje redova gde je vreme označeno kao '(null)' ili nije u očekivanom formatu
invalid_time_rows = crime_df[crime_df['cmplnt_fr_tm'].str.contains('(null)', na=False)]

# Prikazivanje broja takvih redova
print(f"Broj redova sa vremenom označenim kao '(null)': {invalid_time_rows.shape[0]}")

# Prikazivanje tih redova
print(invalid_time_rows[['cmplnt_fr_dt', 'cmplnt_fr_tm']])


C:\Users\abeli\AppData\Local\Temp\ipykernel_17840\3634471186.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  invalid_time_rows = crime_df[crime_df['cmplnt_fr_tm'].str.contains('(null)', na=False)]


Broj redova sa vremenom označenim kao '(null)': 0
Empty DataFrame
Columns: [cmplnt_fr_dt, cmplnt_fr_tm]
Index: []


In [17]:
# Deleting rows with invalid time from crime_df
crime_df = crime_df[~crime_df['cmplnt_fr_tm'].str.contains('(null)', na=False)]

C:\Users\abeli\AppData\Local\Temp\ipykernel_17840\2277994098.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  crime_df = crime_df[~crime_df['cmplnt_fr_tm'].str.contains('(null)', na=False)]


In [19]:
# Warning: This operation will last for a few minutes due to the large size of the dataset

# First, we merge the columns 'cmplnt_fr_dt' and 'cmplnt_fr_tm' into one column
crime_df['datetime'] = pd.to_datetime(crime_df['cmplnt_fr_dt'] + ' ' + crime_df['cmplnt_fr_tm'])

# Renaming the 'datetime' column to 'ds' for use in Prophet
crime_df.rename(columns={'datetime': 'ds'}, inplace=True)



C:\Users\abeli\AppData\Local\Temp\ipykernel_17840\4155335902.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  crime_df['datetime'] = pd.to_datetime(crime_df['cmplnt_fr_dt'] + ' ' + crime_df['cmplnt_fr_tm'])


In [22]:
display(crime_df.head())
display(crime_df.tail())

,cmplnt_fr_dt,cmplnt_fr_tm,ofns_desc,ds
0,2010-10-27T00:00:00.000,19:30:00,PETIT LARCENY,2010-10-27 19:30:00
1,2021-05-24T00:00:00.000,03:00:00,PETIT LARCENY,2021-05-24 03:00:00
2,2010-02-10T00:00:00.000,12:00:00,PETIT LARCENY,2010-02-10 12:00:00
3,2010-02-09T00:00:00.000,15:00:00,PETIT LARCENY,2010-02-09 15:00:00
4,2010-07-29T00:00:00.000,08:00:00,PETIT LARCENY,2010-07-29 08:00:00


,cmplnt_fr_dt,cmplnt_fr_tm,ofns_desc,ds
6230867,2022-09-22T00:00:00.000,16:07:00,ESCAPE 3,2022-09-22 16:07:00
6230868,2022-08-07T00:00:00.000,00:51:00,ESCAPE 3,2022-08-07 00:51:00
6230869,2022-12-19T00:00:00.000,06:36:00,ESCAPE 3,2022-12-19 06:36:00
6230870,2022-11-26T00:00:00.000,14:15:00,ESCAPE 3,2022-11-26 14:15:00
6230871,2022-05-25T00:00:00.000,22:15:00,ESCAPE 3,2022-05-25 22:15:00


In [23]:
# delete cmplnt_fr_dt and cmplnt_fr_tm columns
crime_df.drop(columns=['cmplnt_fr_dt', 'cmplnt_fr_tm'], inplace=True)
display(crime_df.head())
display(crime_df.tail())

,ofns_desc,ds
0,PETIT LARCENY,2010-10-27 19:30:00
1,PETIT LARCENY,2021-05-24 03:00:00
2,PETIT LARCENY,2010-02-10 12:00:00
3,PETIT LARCENY,2010-02-09 15:00:00
4,PETIT LARCENY,2010-07-29 08:00:00


,ofns_desc,ds
6230867,ESCAPE 3,2022-09-22 16:07:00
6230868,ESCAPE 3,2022-08-07 00:51:00
6230869,ESCAPE 3,2022-12-19 06:36:00
6230870,ESCAPE 3,2022-11-26 14:15:00
6230871,ESCAPE 3,2022-05-25 22:15:00


In [24]:
# make ds column the index and sort the index in ascending order
crime_df.set_index('ds', inplace=True)
crime_df.sort_index(inplace=True)

display(crime_df.head())
display(crime_df.tail())

,ofns_desc
ds,
2010-01-01,RAPE
2010-01-01,SEX CRIMES
2010-01-01,OFF. AGNST PUB ORD SENSBLTY &
2010-01-01,SEX CRIMES
2010-01-01,CRIMINAL MISCHIEF & RELATED OF


,ofns_desc
ds,
2022-12-31 23:30:00,SEX CRIMES
2022-12-31 23:30:00,HARRASSMENT 2
2022-12-31 23:40:00,INTOXICATED & IMPAIRED DRIVING
2022-12-31 23:50:00,GRAND LARCENY
2022-12-31 23:55:00,CRIMINAL MISCHIEF & RELATED OF


In [26]:
# Save the cleaned data to a new csv file
crime_df.to_csv('Resources/cleaned_crime_data.csv')
